# Forecast Table Model1

In [2]:
%%pyspark
sql_query = """
SELECT * 
FROM combined_gold.baseproduct as p
INNER JOIN combined_gold.basesalesinvoice as s
ON p.ProductPrimaryKey = s.SalesInvoiceProductKey
"""

df = spark.sql(sql_query)

In [3]:
# Convert to pandas
df = df.toPandas()

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, LassoCV
from sklearn.metrics import mean_squared_error
import numpy as np
!pip install prophet

## Data Prep

In [5]:
df_salestotal = df[['ProductName','InvoiceDate','SalesTotal']]
df_quantity = df[['ProductName','InvoiceDate','Quantity']]
df_unitcost = df[['ProductName','InvoiceDate','UnitCost']]
df_unitprice = df[['ProductName','InvoiceDate','UnitPrice']]

In [6]:
df_salestotal.dropna(inplace=True)
df_quantity.dropna(inplace=True)
df_unitcost.dropna(inplace=True)
df_unitprice.dropna(inplace=True)

In [7]:
# redefine dtypes for analysis
df_salestotal = df_salestotal.copy() 
df_quantity = df_quantity.copy() 
df_unitcost = df_unitcost.copy() 
df_unitprice = df_unitprice.copy() 


df_salestotal['SalesTotal'] = df_salestotal['SalesTotal'].astype(float)
df_quantity['Quantity'] = df_quantity['Quantity'].astype(float)
df_unitcost['UnitCost'] = df_unitcost['UnitCost'].astype(float)
df_unitprice['UnitPrice'] = df_unitprice['UnitPrice'].astype(float)


In [8]:
# Set to post covid data
df_salestotal = df_salestotal.loc[df_salestotal['InvoiceDate'] > '2022-01-01']
df_quantity = df_quantity.loc[df_quantity['InvoiceDate'] > '2022-01-01']
df_unitcost = df_unitcost.loc[df_unitcost['InvoiceDate'] > '2022-01-01']
df_unitprice = df_unitprice.loc[df_unitprice['InvoiceDate'] > '2022-01-01'] 


In [9]:
# Convert 'InvoiceDate' column to datetime
df_salestotal = df_salestotal.copy()
df_quantity = df_quantity.copy()
df_unitcost = df_unitcost.copy()
df_unitprice = df_unitprice.copy()

df_salestotal['InvoiceDate'] = pd.to_datetime(df_salestotal['InvoiceDate'])
df_quantity['InvoiceDate'] = pd.to_datetime(df_quantity['InvoiceDate'])
df_unitcost['InvoiceDate'] = pd.to_datetime(df_unitcost['InvoiceDate'])
df_unitprice['InvoiceDate'] = pd.to_datetime(df_unitprice['InvoiceDate'])

# Set 'InvoiceDate' as the index
df_salestotal.set_index('InvoiceDate', inplace=True)
df_quantity.set_index('InvoiceDate', inplace=True)
df_unitcost.set_index('InvoiceDate', inplace=True)
df_unitprice.set_index('InvoiceDate', inplace=True)

# Group by 'ProductName' and resample by month, summing up the for each month
monthly_totalsales = df_salestotal.groupby('ProductName').resample('M').sum().reset_index()
monthly_quantity = df_quantity.groupby('ProductName').resample('M')['Quantity'].sum().reset_index()
monthly_unitcost = df_unitcost.groupby('ProductName').resample('M')['UnitCost'].sum().reset_index()
monthly_unitprice = df_unitprice.groupby('ProductName').resample('M')['UnitPrice'].sum().reset_index()

# Rename columns for clarity
monthly_totalsales.columns = ['ProductName', 'Month', 'TotalSales']
monthly_quantity.columns = ['ProductName', 'Month', 'Quantity']
monthly_unitcost.columns = ['ProductName', 'Month', 'UnitCost']
monthly_unitprice.columns = ['ProductName', 'Month', 'UnitPrice']
monthly_revenue = monthly_unitprice.copy()
monthly_revenue['Revenue'] = monthly_unitprice['UnitPrice'] - monthly_unitcost['UnitCost']
monthly_revenue.drop(columns=['UnitPrice'],inplace=True)

In [10]:
# Create a pivot table
# Says 'Pivot Table' but this is just the name of the command 
# it is just a table with rows given by the product name 
# and columns given by the available months where the values are the 
# aggregated sales quantities per month
ds1 = monthly_totalsales.pivot(index='ProductName', columns='Month', values='TotalSales')
ds2 = monthly_quantity.pivot(index='ProductName', columns='Month', values='Quantity')
ds3 = monthly_unitcost.pivot(index='ProductName', columns='Month', values='UnitCost')
ds4 = monthly_unitprice.pivot(index='ProductName', columns='Month', values='UnitPrice')
ds5 = monthly_revenue.pivot(index='ProductName', columns='Month', values='Revenue')

# Fill missing values with 0
ds1 = ds1.fillna(0)
ds2 = ds2.fillna(0)
ds3 = ds3.fillna(0)
ds4 = ds4.fillna(0)
ds5 = ds5.fillna(0)

# ds.set_index('ProductName',inplace=True)
transpose1 = ds1.transpose()
transpose2 = ds2.transpose()
transpose3 = ds3.transpose()
transpose4 = ds4.transpose()
transpose5 = ds5.transpose()

# Change index type
transpose1.index = pd.to_datetime(transpose1.index,format='%Y-%m-%d %H:%M:%S')
transpose2.index = pd.to_datetime(transpose2.index,format='%Y-%m-%d %H:%M:%S')
transpose3.index = pd.to_datetime(transpose3.index,format='%Y-%m-%d %H:%M:%S')
transpose4.index = pd.to_datetime(transpose4.index,format='%Y-%m-%d %H:%M:%S')
transpose5.index = pd.to_datetime(transpose5.index,format='%Y-%m-%d %H:%M:%S')

## Model1 TotalSales

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
product_df = transpose1

# Define forecasting parameters
forecast_horizon = 1  # Forecasting for one month ahead
window_size = 9       # Use the last 9 months for modeling

# # Define target months (from January 2023 to September 2023)
target_months = transpose1.iloc[transpose1.index >= '2023-01-01'].index
additional_months = pd.date_range(start = '2023-08-31',end = '2023-10-31',freq='M')
target_months = target_months.union(additional_months)
# Initialize a list to store results
results = []

# Loop through products
for product in product_df.columns:
    product_series = product_df[product]
    # Extract data for modeling, starting from March 2022
    product_series = product_series.loc['2022-03-01':]
    # Loop through target months
    for target_month in target_months:
        # Select the last 9 months for modeling
        end_date = target_month - pd.DateOffset(months=1)
        start_date = end_date - pd.DateOffset(months=window_size-1)
        training_data = product_series.loc[start_date:end_date]
        
        # Fit a SARIMA model 
        sarima_model = SARIMAX(training_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        sarima_fit = sarima_model.fit(disp=False)
        
        # Make a forecast for the target month
        forecast = sarima_fit.forecast(steps=forecast_horizon)
        
        # Get the actual value for the target month
        actual_value = product_series.loc[target_month] if target_month in product_series.index else None
        # Store results for this product and target month
        results.append({'Product': product, 'TargetMonth': target_month, 'ActualSalesTotal': actual_value, 'ForecastSalesTotal': forecast.values[0]})

# Create a pandas DataFrame from the results
salestotal_df = pd.DataFrame(results)
# salestotal_df['SalesTotalModel'] = 'Model1'


## Model1 Quantity

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
product_df = transpose2

# Define forecasting parameters
forecast_horizon = 1  # Forecasting for one month ahead
window_size = 9       # Use the last 9 months for modeling

# # Define target months (from January 2023 to September 2023)
target_months = transpose2.iloc[transpose2.index >= '2023-01-01'].index
additional_months = pd.date_range(start = '2023-08-31',end = '2023-10-31',freq='M')
target_months = target_months.union(additional_months)
# Initialize a list to store results
results = []

# Loop through products
for product in product_df.columns:
    product_series = product_df[product]
    # Extract data for modeling, starting from March 2022
    product_series = product_series.loc['2022-03-01':]
    # Loop through target months
    for target_month in target_months:
        # Select the last 9 months for modeling
        end_date = target_month - pd.DateOffset(months=1)
        start_date = end_date - pd.DateOffset(months=window_size-1)
        training_data = product_series.loc[start_date:end_date]
        
        # Fit a SARIMA model 
        sarima_model = SARIMAX(training_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        sarima_fit = sarima_model.fit(disp=False)
        
        # Make a forecast for the target month
        forecast = sarima_fit.forecast(steps=forecast_horizon)
        
        # Get the actual value for the target month
        actual_value = product_series.loc[target_month] if target_month in product_series.index else None
        # Store results for this product and target month
        results.append({'Product': product, 'TargetMonth': target_month, 'ActualQuantity': actual_value, 'ForecastQuantity': forecast.values[0]})

# Create a pandas DataFrame from the results
quantity_df = pd.DataFrame(results)
# quantity_df['QuantityModel'] = 'Model1'


## Model1 UnitCost

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
product_df = transpose3

# Define forecasting parameters
forecast_horizon = 1  # Forecasting for one month ahead
window_size = 9       # Use the last 9 months for modeling

# # Define target months (from January 2023 to September 2023)
target_months = transpose3.iloc[transpose3.index >= '2023-01-01'].index
additional_months = pd.date_range(start = '2023-08-31',end = '2023-10-31',freq='M')
target_months = target_months.union(additional_months)
# Initialize a list to store results
results = []

# Loop through products
for product in product_df.columns:
    product_series = product_df[product]
    # Extract data for modeling, starting from March 2022
    product_series = product_series.loc['2022-03-01':]
    # Loop through target months
    for target_month in target_months:
        # Select the last 9 months for modeling
        end_date = target_month - pd.DateOffset(months=1)
        start_date = end_date - pd.DateOffset(months=window_size-1)
        training_data = product_series.loc[start_date:end_date]
        
        # Fit a SARIMA model 
        sarima_model = SARIMAX(training_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        sarima_fit = sarima_model.fit(disp=False)
        
        # Make a forecast for the target month
        forecast = sarima_fit.forecast(steps=forecast_horizon)
        
        # Get the actual value for the target month
        actual_value = product_series.loc[target_month] if target_month in product_series.index else None
        # Store results for this product and target month
        results.append({'Product': product, 'TargetMonth': target_month, 'ActualUnitCost': actual_value, 'ForecastUnitCost': forecast.values[0]})

# Create a pandas DataFrame from the results
unitcost_df = pd.DataFrame(results)
# unitcost_df['UnitCostModel'] = 'Model1'

## Model1 UnitPrice

In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
product_df = transpose4

# Define forecasting parameters
forecast_horizon = 1  # Forecasting for one month ahead
window_size = 9       # Use the last 9 months for modeling

# # Define target months (from January 2023 to September 2023)
target_months = transpose4.iloc[transpose4.index >= '2023-01-01'].index
additional_months = pd.date_range(start = '2023-08-31',end = '2023-10-31',freq='M')
target_months = target_months.union(additional_months)
# Initialize a list to store results
results = []

# Loop through products
for product in product_df.columns:
    product_series = product_df[product]
    # Extract data for modeling, starting from March 2022
    product_series = product_series.loc['2022-03-01':]
    # Loop through target months
    for target_month in target_months:
        # Select the last 9 months for modeling
        end_date = target_month - pd.DateOffset(months=1)
        start_date = end_date - pd.DateOffset(months=window_size-1)
        training_data = product_series.loc[start_date:end_date]
        
        # Fit a SARIMA model 
        sarima_model = SARIMAX(training_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        sarima_fit = sarima_model.fit(disp=False)
        
        # Make a forecast for the target month
        forecast = sarima_fit.forecast(steps=forecast_horizon)
        
        # Get the actual value for the target month
        actual_value = product_series.loc[target_month] if target_month in product_series.index else None
        # Store results for this product and target month
        results.append({'Product': product, 'TargetMonth': target_month, 'ActualUnitPrice': actual_value, 'ForecastUnitPrice': forecast.values[0]})

# Create a pandas DataFrame from the results
unitprice_df = pd.DataFrame(results)
# unitprice_df['UnitPriceModel'] = 'Model1'

## Model1 Revenue

In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
product_df = transpose5

# Define forecasting parameters
forecast_horizon = 1  # Forecasting for one month ahead
window_size = 9       # Use the last 9 months for modeling

# # Define target months (from January 2023 to September 2023)
target_months = transpose5.iloc[transpose5.index >= '2023-01-01'].index
additional_months = pd.date_range(start = '2023-08-31',end = '2023-10-31',freq='M')
target_months = target_months.union(additional_months)
# Initialize a list to store results
results = []

# Loop through products
for product in product_df.columns:
    product_series = product_df[product]
    # Extract data for modeling, starting from March 2022
    product_series = product_series.loc['2022-03-01':]
    # Loop through target months
    for target_month in target_months:
        # Select the last 9 months for modeling
        end_date = target_month - pd.DateOffset(months=1)
        start_date = end_date - pd.DateOffset(months=window_size-1)
        training_data = product_series.loc[start_date:end_date]
        
        # Fit a SARIMA model 
        sarima_model = SARIMAX(training_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        sarima_fit = sarima_model.fit(disp=False)
        
        # Make a forecast for the target month
        forecast = sarima_fit.forecast(steps=forecast_horizon)
        
        # Get the actual value for the target month
        actual_value = product_series.loc[target_month] if target_month in product_series.index else None
        # Store results for this product and target month
        results.append({'Product': product, 'TargetMonth': target_month, 'ActualRevenue': actual_value, 'ForecastRevenue': forecast.values[0]})

# Create a pandas DataFrame from the results
revenue_df = pd.DataFrame(results)
# unitprice_df['UnitPriceModel'] = 'Model1'

# Create final dataframe with all forecasting info

In [16]:
dfs1 = [salestotal_df,quantity_df,unitcost_df,unitprice_df,revenue_df]
final_df1 = pd.concat(dfs1,axis='columns')
final_df1 = final_df1.loc[:,~final_df1.columns.duplicated()]
final_df1['Model'] = 'Model1'
final_df1

In [17]:
final_df1.fillna(0,inplace=True)

## Create template for D365

In [18]:
# final_df3.to_csv('forecastmodel3.txt',sep='\t',index=False)

In [19]:
# container_directory = ""
# %cd $container_directory

In [20]:
# import os
# current_directory = os.getcwd()
# current_directory

In [21]:
# %ls

In [22]:
# file_path = '/mnt/var/hadoop/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1695909286640_0004/container_1695909286640_0004_01_000001/forecastmodel3.txt'
# with open(file_path,'r') as file:
#     file_contents = file.read()
# print(file_contents)

In [23]:
# try:
#     os.remove(file_path)
#     print(f"File '{file_path}' has been deleted.")
# except FileNotFoundError:
#     print(f"File '{file_path}' does not exist.")
# except Exception as e:
#     print(f"An error occurred '{str(e)}")


# Import final dataframe into combined_gold

In [24]:
# turn pd dataframe into sparks df then into a view that sql can take
from pyspark.sql.functions import col


spark_df = spark.createDataFrame(final_df1)


spark_df = spark_df.select([col(c).alias(
        c.replace( '(', '')
        .replace( ')', '')
        .replace( ',', '')
        .replace( ';', '')
        .replace( '{', '')
        .replace( '}', '')
        .replace( '\n', '')
        .replace( '\t', '')
        .replace( ' ', '_')
    ) for c in spark_df.columns])

spark_df.createOrReplaceTempView("temp_view")

In [25]:
#setting spark configuration 

spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")

In [26]:
%%sql

select * from temp_view

In [27]:
%%sql

DROP Table IF Exists combined_gold.forecastmodel1;

CREATE TABLE combined_gold.forecastmodel1
   USING DELTA;

 INSERT INTO combined_gold.forecastmodel1

 SELECT *
 FROM temp_view

In [28]:
%%sql

SELECT * FROM combined_gold.forecastmodel1;